<a href="https://colab.research.google.com/github/CDU-data-science-team/zero-shot/blob/feature-Huggingface_transformer/hf_T_training_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets #needed for loading metric

     |████████████████████████████████| 2.6 MB 7.9 MB/s 
     |████████████████████████████████| 636 kB 53.2 MB/s 
     |████████████████████████████████| 895 kB 41.9 MB/s 
     |████████████████████████████████| 3.3 MB 45.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 542 kB 7.6 MB/s 
     |████████████████████████████████| 118 kB 16.3 MB/s 
     |████████████████████████████████| 243 kB 15.5 MB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
#? not sure this is needed. all i needed is means of evaluating
#from datasets import load_metric    # to instantiate a metric 
# metric = load_metric('Accuracy')
#ex
# fake_preds = np.random.randint(0, 2, size=(64,))
# fake_labels = np.random.randint(0, 2, size=(64,))
# metric.compute(predictions=fake_preds, references=fake_labels)

##Import needed library

In [ ]:
import transformers  
print(transformers.__version__) #print th tranformer version
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer    #the API for training model in transormer since PyTorch does not provide a training loop
from sklearn.model_selection import train_test_split

4.9.0


##Loading and split the dataset

In [ ]:
import pandas as pd
df = pd.read_csv('new_data.csv', usecols=['label', 'feedback'])   #, nrows=5)

#fill missing value has this causes runtime error while fiting the model 
print(df.isna().sum())
df['feedback'].fillna('Nothing', inplace = True)
df.isna().sum()

label       0
feedback    1
dtype: int64


label       0
feedback    0
dtype: int64

In [ ]:
#Split the dataset
seed = 0
candidate_labels = df.label.unique()
num_label = len(candidate_labels)

train_feedback, test_feedback, train_label, test_label = train_test_split(df['feedback'], 
                                                                          df['label'], test_size=0.4, random_state=seed)
val_feedback, test_feedback, val_label, test_label = train_test_split(test_feedback, 
                                                                      test_label, test_size=0.5, random_state=seed)

#trying this to test if the pipeline will work
# train_feedback, test_feedback = train_test_split(df, test_size=0.4, random_state=seed)
# val_feedback, test_feedback = train_test_split(test_feedback, test_size=0.5, random_state=seed)

In [ ]:
# #seperate the labels from the feedbacks
# sequence = df.feedback
# candidate_labels = df.label.unique()

# # split the sequence
# train = sequence.iloc[:701]
# eval = sequence.iloc[701:1001]
# test = sequence.iloc[1001:1301]

# df.reset_index(inplace=True, drop=True)
# print(train.tail())
# eval.head()
# test.tail()

In [ ]:
print(train_feedback.shape)
print(val_feedback.shape)
test_feedback.shape

(5574, 2)
(1858, 2)


(1859, 2)

#Preprocessing the data

In [ ]:
model_checkpoint = 'bert-base-cased'
#model_checkpoint = "distilbert-base-uncased"

#instantiate our tokenizer. 
try:
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, num_labels=num_label, use_fast=True) 
except:
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, num_labels=len(candidate_labels))   # remove  use_fast=True if error is thrown from above

#Define function for tokenising
def tokenize_function(examples):
    return tokenizer('examples', padding="max_length", truncation=True)

#preprocess all the datasets in one line on code
tokenized_train, tokenized_val, tokenized_test = [list(map(tokenize_function, data)) for data in [train_feedback, val_feedback, test_feedback]]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

# Define our model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_label)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

#Fine-tuning the model

### Instantiate a TrainingArguments 
to hold all the hyperparameters we can tune for the Trainer


In [ ]:
metric_name = 'accuracy'

#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics  #!define compute_metrics function
# )batch_size = 10

#this uses all the defaults
args = TrainingArguments(
    "train-model",    # <output_dir: str> required to save the checkpoints of the model, all other arguments are optional
    evaluation_strategy = "epoch",  #to fine-tune your model and regularly report the evaluation metrics at the end of each epoch 
    # learning_rate=2e-5,
    # per_device_train_batch_size=batch_size,   # batch size per device during training
    # per_device_eval_batch_size=batch_size,    # batch size for evaluation
    # num_train_epochs=5,                       # total number of training epochs
    # weight_decay=0.01,
    # load_best_model_at_end=True,
    # metric_for_best_model=metric_name,
)

### Define a function for evaluating the model
needs to takes predictions and labels (grouped in a namedtuple called EvalPrediction)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

### Instantiate a trainer

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics  #!define compute_metrics function
)

# trainer = Trainer(
#     model,
#     args,
#     train_dataset=train_feedback,
#     eval_dataset=val_feedback,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics  #!define compute_metrics function
# )

### Train the model 
to do the actual fine tunning of the model

In [ ]:
trainer.train()   

***** Running training *****
  Num examples = 2
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3


KeyError: ignored

In [ ]:
trainer.evaluate()


#predictions = trainer.predict(tokenized_test)["logits"]

***** Running Evaluation *****
  Num examples = 2
  Batch size = 8


AttributeError: ignored

#Hyperparameter search

Ignore this section for now

The hyperparameter_search method returns a BestRun objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
#Hyperparameter search (The Trainer supports hyperparameter search using optuna or Ray Tune. )
# ! pip install optuna
# ! pip install ray[tune]    #run either

#needed because the Trainer will run several trainings
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")   #ustomize the search space by passing a hp_space argument
best_run

In [ ]:
#to reproduce the best training, just set the hyperparameters in your TrainingArgument before creating a Trainer:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

In [ ]:
#EvalPrediction
from datasets import load_metric
metric = load_metric('glue', actual_task)
metric.compute(predictions=predictions, references=labels)